To fazendo alguns testes separados aqui nesse notebook, vou tentar fazer esse teste de exemplo que tem no pdf novo que subi, que tá no moodle. Acho que as duas principais coisas que temos que resolver é conseguir executar uma arvore de decisão simples com esses dados de benchmark e fazer um cross validation com algum outro tipo de dado, pq o meu do exercício anterior não deu certo.

Vou seguir fazendo esse exemplo aqui, depois se puder sobe teu trabalho de crossvalidation pra gente já começar a ajustar caso não esteja fazendo aquela divisão estratificada que falamos aquele dia. 

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from __future__ import print_function

dataset = pd.read_csv("C:/Users/renan/Desktop/UFRGS/Machine Learning/Trabalho1/random-forests-1/dataset.csv") 

dir(dataset)

print ('Banco com %d amostras e %d colunas\n' % dataset.shape)

print ('Tabela sumarizando as colunas do banco\n')
dataset.describe()

Banco com 14 amostras e 5 colunas

Tabela sumarizando as colunas do banco



,Tempo,Temperatura,Umidade,Ventoso,Joga
count,14,14,14,14,14
unique,3,3,2,2,2
top,Chuvoso,Amena,Normal,Falso,Sim
freq,5,6,7,8,9


In [2]:
def entropy(dataset):

    """
    Calculo da entropia do conjunto de dados.
    Recebe como parâmetro o dataset. 
    """

    Class = dataset.keys()[-1]   #Define a variável alvo como Class
    entropy = 0
    values = dataset[Class].unique()
    for value in values:
        fraction = dataset[Class].value_counts()[value]/len(dataset[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

#Teste dos valores gerados pela função 
teste_entropia = entropy(dataset)
print (teste_entropia)

0.9402859586706311


In [3]:
result = entropy(dataset)
print (result)

0.9402859586706311


In [4]:
def find_entropy_attribute(df,attribute):
  Class = df.keys()[-1]   #To make the code generic, changing target variable class name
  target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
  variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)


def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]
  
  
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)


def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(df)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Eat'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree